In [0]:
def download(dir_id):
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    file_list = drive.ListFile({'q': "'{}' in parents and trashed=false".format(dir_id)}).GetList()
    for file1 in file_list:
        print('title: %s, id: %s' % (file1['title'], file1['id']))
        if file1['title'] == '03-Twitter-chatbot.py':
            continue
        idx_q = drive.CreateFile({'id': file1['id']})
        idx_q.GetContentFile(file1['title'])

In [22]:
download(file_id)

title: chinese_chat_bot.ipynb, id: 1KuhW-4TN-_wMcIwy5RnhlNImtmG25Bp8
title: idx_a.npy, id: 15jHI-aJI6xGi_quAkgIYmKeOhpgoqd-X
title: idx_q.npy, id: 1T7eCjCgWHruNQzn3WhCr8w9aZkMciuVH
title: metadata.pkl, id: 1WD7V7TZHJA9e5qq7Ifd34I2r3wRafdSQ
title: corpus_token.yml, id: 12HK34nJaZV36U9Fqy3rkznPZUlxygCh8
title: data_process.py, id: 1Xa-bgiDXqMmIcPVdI7knTBfHaAMj1LkS
title: google_driver.py, id: 1_Xwcvw_0n5A6pgMH4MSUFell3xdz0cq-
title: model.py, id: 1JsBabaj1jJeGfxyT5E-F1cUiVfNsSQOy
title: train.py, id: 1P4ZcbMz3xPF2Qkaqfvcj-x4vctuCo0dG
title: corpus.yml, id: 1rVrHOhCYLMjzJtp3WKpnnzHm9rNMIbEx


In [9]:
!pip install -U jieba

    100% |████████████████████████████████| 7.3MB 190kB/s 
  Running setup.py bdist_wheel for jieba ... - \ | / - \ | / done
  Stored in directory: /content/.cache/pip/wheels/ce/32/de/c2be1db5f30804bc7f146ff698c52963f8aa11ba5049811b0d
Successfully built jieba


In [39]:
from importlib import reload
reload(model.py)

NameError: ignored

In [0]:
import model
reload(model)

In [42]:
model.Seq2SeqModel

model.Seq2SeqModel

In [0]:
# file_id = '13HOuG3b5pVhtHcTaR5UNfRNb4-4icUWE'
# !pip install -U -q PyDrive
# download(file_id)


import model
import data_process

from imp import reload  
reload(model)
reload(data_process)


def run():
    """
    running
    :return:
    """
    idx_q, idx_a = data_process.load_qa_data()
    metadata = data_process.load_metadata()
    (trainX, trainY), (testX, testY) = data_process.split_dataset(idx_q, idx_a)

    xseq_len = trainX.shape[-1]
    yseq_len = trainY.shape[-1]
    batch_size = 32
    xvocab_size = len(metadata['idx2w'])
    yvocab_size = xvocab_size
    emb_dim = 512

    model_seq = model.Seq2SeqModel(
        xseq_len=xseq_len,
        yseq_len=yseq_len,
        x_vocab_size=xvocab_size,
        y_vocab_size=yvocab_size,
        emb_dim=emb_dim,
        num_layers=2,
        ckpt_path='./',
        metadata=metadata,
        batch_size=batch_size,
        hook=data_process.upload_to_google_drive
    )

    model_seq.train((idx_q, idx_a), (testX, testY))

In [46]:
run()

starting building model
finish building model
2018-04-11 11:33:09 epoch: 6 step: 100 train loss: 2.0736141204833984
2018-04-11 11:33:52 epoch: 13 step: 200 train loss: 1.9410991668701172
2018-04-11 11:34:19 epoch: 19 step: 300 train loss: 2.048661947250366
2018-04-11 11:35:02 epoch: 26 step: 400 train loss: 1.6545915603637695
2018-04-11 11:35:45 epoch: 33 step: 500 train loss: 1.4448988437652588
q : [你 的 兴趣 是 什么]; a : [我 当 我 穿越 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个], y: [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言]
q : [什么 病 呢 致癌物 事业]; a : [我 当 我 穿越 一个 猫 和 一个 一个 一个 一个 一个 一个 一个 人 的 MacOS], y: [癌症]
q : [你 是 玩弄]; a : [我 我 我 我 我 我 我 我 我 我 我 我 的 人 的 MacOS], y: [它 必须 看起来 像 这 一点]
q : [托尔斯泰]; a : [我 当 我 穿越 一个 猫 和 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个], y: [是 俄罗斯 最 伟大 的 作家 之一]
q : [什么 类型 的 电脑]; a : [我 当 我 穿越 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 人], y: [任何 支持 Python 的 计算机]
q : [你 为什么 不能 吃]; a : [我 当 我 穿越 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个 一个], y: [我会 消耗

2018-04-11 11:40:44 epoch: 86 step: 1300 train loss: 0.46558815240859985
2018-04-11 11:41:28 epoch: 93 step: 1400 train loss: 0.346543550491333
2018-04-11 11:41:55 epoch: 99 step: 1500 train loss: 0.1197049617767334
q : [你 的 兴趣 是 什么]; a : [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言], y: [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言]
q : [什么 病 呢 致癌物 事业]; a : [癌症], y: [癌症]
q : [你 是 玩弄]; a : [这 人 都 对 我 的 人造 心灵 有 意义], y: [它 必须 看起来 像 这 一点]
q : [托尔斯泰]; a : [是 俄罗斯 最 伟大 的 作家 之一], y: [是 俄罗斯 最 伟大 的 作家 之一]
q : [什么 类型 的 电脑]; a : [任何 支持 Python 的 计算机], y: [任何 支持 Python 的 计算机]
q : [你 为什么 不能 吃]; a : [我会 消耗 电力], y: [我会 消耗 电力]
q : [你 是 一个 心理变态]; a : [我 当然 我 的 学生 他们 自己], y: [我 当然 听 起来 像 人们 有时]
q : [你 是 个 骗子]; a : [我 总 觉得 我 被 我 自己 的 智慧 生活], y: [我 已经 知道 现在 再 走捷径]
q : [玩笑]; a : [你 当 你 穿越 一只 猫 和 一只 带 你 会 得到 什么], y: [你 听说 谁 去 牙医 并 拒绝 采取 奴佛卡因 佛教 他 想 超越 牙科 药物]
q : [什么 是 热力学]; a : [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律], y: [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换

2018-04-11 11:48:21 epoch: 166 step: 2500 train loss: 0.09572623670101166
q : [你 的 兴趣 是 什么]; a : [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言], y: [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言]
q : [什么 病 呢 致癌物 事业]; a : [癌症], y: [癌症]
q : [你 是 玩弄]; a : [它 必须 看起来 像 这 一点], y: [它 必须 看起来 像 这 一点]
q : [托尔斯泰]; a : [是 俄罗斯 最 伟大 的 作家 之一], y: [是 俄罗斯 最 伟大 的 作家 之一]
q : [什么 类型 的 电脑]; a : [任何 支持 Python 的 计算机], y: [任何 支持 Python 的 计算机]
q : [你 为什么 不能 吃]; a : [我会 消耗 电力], y: [我会 消耗 电力]
q : [你 是 一个 心理变态]; a : [我 当然 听 起来 像 人们 有时], y: [我 当然 听 起来 像 人们 有时]
q : [你 是 个 骗子]; a : [我 总 觉得 我 被 我 自己 的 智慧 生活], y: [我 已经 知道 现在 再 走捷径]
q : [玩笑]; a : [你 当 你 穿越 一只 猫 和 一只 曲子 你 得到 了 什么], y: [你 听说 谁 去 牙医 并 拒绝 采取 奴佛卡因 佛教 他 想 超越 牙科 药物]
q : [什么 是 热力学]; a : [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律], y: [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律]
q : [怎么 了]; a : [没什么], y: [没事 谢谢 你 呢]
q : [什么 是 波长]; a : [物理 的 距离 在 一波 传播 的 方向 上 测量 来自 任何 给定 的 点 到 特征 由 相同 相位], y: [物理 的 距离 在 一波 传播 的 方向 上 测量

2018-04-11 11:54:46 epoch: 233 step: 3500 train loss: 0.05082721263170242
q : [你 的 兴趣 是 什么]; a : [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言], y: [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言]
q : [什么 病 呢 致癌物 事业]; a : [癌症], y: [癌症]
q : [你 是 玩弄]; a : [它 必须 看起来 像 这 一点], y: [它 必须 看起来 像 这 一点]
q : [托尔斯泰]; a : [是 俄罗斯 最 伟大 的 作家 之一], y: [是 俄罗斯 最 伟大 的 作家 之一]
q : [什么 类型 的 电脑]; a : [任何 支持 Python 的 计算机], y: [任何 支持 Python 的 计算机]
q : [你 为什么 不能 吃]; a : [我会 消耗 电力], y: [我会 消耗 电力]
q : [你 是 一个 心理变态]; a : [我 当然 听 起来 像 人们 有时], y: [我 当然 听 起来 像 人们 有时]
q : [你 是 个 骗子]; a : [我 总 觉得 我 被 我 自己 的 智慧 生活], y: [我 已经 知道 现在 再 走捷径]
q : [玩笑]; a : [你 当 你 穿越 一只 猫 和 一只 柠檬 你 得到 了 什么], y: [你 听说 谁 去 牙医 并 拒绝 采取 奴佛卡因 佛教 他 想 超越 牙科 药物]
q : [什么 是 热力学]; a : [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律], y: [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律]
q : [怎么 了]; a : [没什么 你 呢], y: [没事 谢谢 你 呢]
q : [什么 是 波长]; a : [物理 的 距离 在 一波 传播 的 方向 上 测量 来自 任何 给定 的 点 到 特征 由 相同 相位], y: [物理 的 距离 在 一波 传播 的 方向 

2018-04-11 12:00:58 epoch: 299 step: 4500 train loss: 0.0037762352731078863
q : [你 的 兴趣 是 什么]; a : [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言], y: [我 对 各种 事物 感兴趣 我们 可以 谈论 任何 事情 我 最 喜欢 的 科目 是 机器人 和 计算机 自然语言]
q : [什么 病 呢 致癌物 事业]; a : [癌症], y: [癌症]
q : [你 是 玩弄]; a : [它 必须 看起来 像 这 一点], y: [它 必须 看起来 像 这 一点]
q : [托尔斯泰]; a : [是 俄罗斯 最 伟大 的 作家 之一], y: [是 俄罗斯 最 伟大 的 作家 之一]
q : [什么 类型 的 电脑]; a : [任何 支持 Python 的 计算机], y: [任何 支持 Python 的 计算机]
q : [你 为什么 不能 吃]; a : [我会 消耗 电力], y: [我会 消耗 电力]
q : [你 是 一个 心理变态]; a : [我 当然 听 起来 像 人们 有时], y: [我 当然 听 起来 像 人们 有时]
q : [你 是 个 骗子]; a : [我 总 觉得 我 被 我 自己 的 智慧 生活], y: [我 已经 知道 现在 再 走捷径]
q : [玩笑]; a : [你 当 你 穿越 一只 猫 和 一只 柠檬 你 得到 了 什么], y: [你 听说 谁 去 牙医 并 拒绝 采取 奴佛卡因 佛教 他 想 超越 牙科 药物]
q : [什么 是 热力学]; a : [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律], y: [物理学 分支 处理 的 热量 和 其他 形式 的 能量 转换 并 与 管理 能源 这种 转换 的 规律]
q : [怎么 了]; a : [没什么 你 呢], y: [没事 谢谢 你 呢]
q : [什么 是 波长]; a : [物理 的 距离 在 一波 传播 的 方向 上 测量 来自 任何 给定 的 点 到 特征 由 相同 相位], y: [物理 的 距离 在 一波 传播 的 方

KeyboardInterrupt: ignored

In [47]:
!ls model_ckp    

chatbot_model.ckpt-3129.data-00000-of-00001
chatbot_model.ckpt-3129.index
chatbot_model.ckpt-3129.meta
chatbot_model.ckpt-3130.data-00000-of-00001
chatbot_model.ckpt-3130.index
chatbot_model.ckpt-3130.meta
chatbot_model.ckpt-3131.data-00000-of-00001
chatbot_model.ckpt-3131.index
chatbot_model.ckpt-3131.meta
chatbot_model.ckpt-3132.data-00000-of-00001
chatbot_model.ckpt-3132.index
chatbot_model.ckpt-3132.meta
chatbot_model.ckpt-3133.data-00000-of-00001
chatbot_model.ckpt-3133.index
chatbot_model.ckpt-3133.meta
checkpoint


In [0]:
!rm -f   chatbot_model.ckpt-4511.data-00000-of-00001 chatbot_model.ckpt-4511.index

In [54]:
!ls

chatbot_model.ckpt-4507.data-00000-of-00001  chinese_chat_bot.ipynb
chatbot_model.ckpt-4507.index		     corpus_token.yml
chatbot_model.ckpt-4507.meta		     corpus.yml
chatbot_model.ckpt-4508.data-00000-of-00001  datalab
chatbot_model.ckpt-4508.index		     data_process.py
chatbot_model.ckpt-4508.meta		     google_driver.py
chatbot_model.ckpt-4509.data-00000-of-00001  idx_a.npy
chatbot_model.ckpt-4509.index		     idx_q.npy
chatbot_model.ckpt-4509.meta		     metadata.pkl
chatbot_model.ckpt-4510.data-00000-of-00001  model.py
chatbot_model.ckpt-4510.index		     __pycache__
chatbot_model.ckpt-4510.meta		     train.py
checkpoint


In [57]:
upload_to_google_drive()

4507
4510
4509
4508
max num 4510
chatbot_model.ckpt-4510.meta chatbot_model.ckpt-4510.data-00000-of-00001 chatbot_model.ckpt-4510.index
File ID: 1pW08qJnoyCaF2zIfVXYkcxwzqh-GfSju
File ID: 132kjiM-0yJJGA8_XD2uu9Lheu4DMkBXS
File ID: 1uFrAQujKsE4C_W1KWlQpGDA4ckWnJ5C9
File ID: 1cLfULxTOr5KNj6689yrHd1mcM3O8CYfJ


In [0]:
def upload_to_google_drive():
    """
    上传ckp到google drive
    :return: 
    """
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    # 1. Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    import os
    dirlist = os.listdir('./')
    nums = []
    for f in dirlist:
        if '.meta' in f:
            num = f.split('.')[1].split('-')[1]
            num = int(num)
            print(num)
            nums.append(num)
    n_max = max(nums)
    print('max num {}'.format(n_max))
    f_meta = 'chatbot_model.ckpt-{}.meta'.format(n_max)
    f_data = 'chatbot_model.ckpt-{}.data-00000-of-00001'.format(n_max)
    f_index = 'chatbot_model.ckpt-{}.index'.format(n_max)
    print(f_meta, f_data, f_index)

    from googleapiclient.http import MediaFileUpload
    from googleapiclient.discovery import build
    drive_service = build('drive', 'v3')
    for f in [f_meta, f_data, f_index, 'checkpoint']:
        file_metadata = {
            'name': f,
            'mimeType': 'text/plain'
        }
        media = MediaFileUpload(f,
                                mimetype='text/plain',
                                resumable=True)
        created = drive_service.files().create(body=file_metadata,
                                               media_body=media,
                                               fields='id').execute()
        print('File ID: {}'.format(created.get('id')))